# Nephrotic syndrome computable phenotype
This tutorial demonstrates three different ways to use the Nephrotic Syndrome Computable Phenotype Knowledge Object:

- using the built-in command-line interface (CLI) service
- using the built-in web API service
- using the knowledge object's code and modules directly in your own code



## Prepare the environment
First, let's set up the environment by installing the Knowledge Object as a package and downloading the sample input file to the Google Colab directory as a local file.

### 1. Install the package (knowledge object)

In [1]:
!pip install git+https://github.com/kgrid-lab/nephroticsyndrome-computablephenotype.git


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/kgrid-lab/nephroticsyndrome-computablephenotype.git to c:\users\faridsei\appdata\local\temp\pip-req-build-jn5mx7ug
  Resolved https://github.com/kgrid-lab/nephroticsyndrome-computablephenotype.git to commit dd55f47889a956b579734f0a253f183f7c98fa89
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
                                              0.0/57.0 kB ? eta -:--:--
     ---------------------                    30.7/57.0 kB 1.3 MB/s eta 0:00:01
     ----------------------------------     51.2/57.0 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 57.0/57.0 kB 424.8 kB/s eta 0:00:00
   

  Running command git clone --filter=blob:none --quiet https://github.com/kgrid-lab/nephroticsyndrome-computablephenotype.git 'C:\Users\faridsei\AppData\Local\Temp\pip-req-build-jn5mx7ug'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
python-activator 0.1.0 requires fastapi<0.99.0,>=0.98.0, but you have fastapi 0.97.0 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### 2. Download the sample input file from GitHub to Google Colab

In [2]:
!curl -o sample_input.json https://raw.githubusercontent.com/kgrid-lab/nephroticsyndrome-computablephenotype/refs/heads/main/input_test_data/sample_input.json
!cat sample_input.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 12407  100 12407    0     0  13545      0 --:--:-- --:--:-- --:--:-- 13634
'cat' is not recognized as an internal or external command,
operable program or batch file.


## Use the **command-line interface service** to classify patients.
The following command calls `nephroticsyndrome-computablephenotype` and passes the sample input file for processing. It will output the patients classified as inclusions.


In [4]:
!nephroticsyndrome-computablephenotype sample_input.json  

{
    "num of inclusion patients": 2,
    "inclusion patients": [
        {
            "patId": "1",
            "age": 30,
            "Anchor_Admit_Date": "10/10/2021",
            "Anchor_Discharge_Date": "10/10/2021",
            "Anchor_EncounterID": "2"
        },
        {
            "patId": "2",
            "age": 14,
            "Anchor_Admit_Date": "5/12/2015",
            "Anchor_Discharge_Date": "5/12/2015",
            "Anchor_EncounterID": "23"
        }
    ]
}


## Use the **web app service** to classify patients
This section demonstrates how to run the web API service of the Knowledge Object and send a POST request to process data using the API from a different client application.

### 1. Run the web API service in the background
First, let's run the web API service in the background so that we can send POST requests to it from this notebook.

In [2]:
# import os
# os.system('nohup uvicorn nephroticsyndrome_computablephenotype.api:app --host 0.0.0.0 --port 8000 2>&1 &')
import subprocess

# Start the Uvicorn server as a background process
process =subprocess.Popen(
    ["uvicorn", "nephroticsyndrome_computablephenotype.api:app", "--host", "0.0.0.0", "--port", "8000"]
)

### 2. Send a post request with the `sample_input.json` as the body of the request
Now, let's write a simple client application that sends a POST request using the sample input file and displays the result of the classification.

In [5]:
import requests
import json

with open('sample_input.json', 'r') as file:
    json_payload = json.load(file)
response = requests.post('http://localhost:8000/classification1', json=json_payload)

# Print the response from the server
print(response.status_code)
print(json.dumps(response.json(), indent=4))

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /classification1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001F7CA06FB50>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

### 3. Stop web API service 
Stop the server running in the background to release the port

In [4]:
process.terminate()

## Use the knowledge object **directly from the code** to classify patients
This section demonstrates how the knowledge embedded in this Knowledge Object can be used directly in the code of a client application. Since the `nephroticsyndrome_computablephenotype` Knowledge Object was installed earlier using the `pip` command, the client application can import it as a package and access its modules and functions. The following code loads the contents of the sample input file into an array of `Patient` objects. It then calls the `process_patient_list` function and passes the array of `Patient` objects to it. This function outputs the classification results.

In [ ]:
from nephroticsyndrome_computablephenotype.classification_algorithm import process_patient_list
from nephroticsyndrome_computablephenotype.datatypes import  Patient

with open('sample_input.json') as f:
    data = json.load(f)
patients = [Patient(**person_data) for person_data in data]
print(json.dumps(process_patient_list(patients), indent=4))

{
    "num of inclusion patients": 2,
    "inclusion patients": [
        {
            "patId": "1",
            "age": 30,
            "Anchor_Admit_Date": "10/10/2021",
            "Anchor_Discharge_Date": "10/10/2021",
            "Anchor_EncounterID": "2"
        },
        {
            "patId": "2",
            "age": 14,
            "Anchor_Admit_Date": "5/12/2015",
            "Anchor_Discharge_Date": "5/12/2015",
            "Anchor_EncounterID": "23"
        }
    ]
}
